# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list =[]

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path = glob.glob(os.path.join(root,'*'))
    #print(file_path)
    file_path_list.append(file_path)
print(file_path)
print(file_path_list[0])


/home/workspace
['/home/workspace/event_data/.ipynb_checkpoints/2018-11-01-events-checkpoint.csv']
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list[0]:
    print (f)

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


/home/workspace/event_data/2018-11-27-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-07-events.csv
/home/workspace/event_data/2018-11-09-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-20-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-21-events.csv
/home/workspace/event_data/2018-11-10-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-02-events.csv
/home/workspace/event_data/2018-11-28-events.csv
/home/workspace/even

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
data = pd.read_csv('event_datafile_new.csv')

In [6]:
data.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [7]:
data.shape

(6820, 11)

In [8]:
data[data.sessionId==338]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
441,Pixies,Ava,F,1,Robinson,89.36444,free,"New Haven-Milford, CT",338,Build High,50
442,The Roots / Jack Davey,Ava,F,2,Robinson,155.95057,free,"New Haven-Milford, CT",338,Atonement,50
443,Mike And The Mechanics,Ava,F,3,Robinson,275.12118,free,"New Haven-Milford, CT",338,A Beggar On A Beach Of Gold,50
444,Faithless,Ava,F,4,Robinson,495.30730,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)


#### Set Keyspace

In [11]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [12]:
query="create table artist_session "
query= query + "( sessionId int, itemInSession int, artist text,song text, length text,  "
query=query + "primary key (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [13]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#query = "select artist,song,length from artist_session where sessionId = 338 and itemInSession = 4"
query = "select artist,song,length from artist_session where sessionId = 139 and itemInSession = 8"
print(query)
try:
    rows=session.execute(query)
    print(rows)
    for row in rows:
        print (row.artist, row.song," - ",row.length)
except Exception as e:
    print(e)


select artist,song,length from artist_session where sessionId = 139 and itemInSession = 8


In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into artist_session ( artist, song, length, sessionId, itemInSession )"
        query = query + "values (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0],line[9],line[5],int(line[8]),int(line[3])))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "select artist,song,length from artist_session where sessionId = 338 and itemInSession = 4"
#query = "select artist,song,length from artist_session where sessionId = 139 and itemInSession = 8"
try:
    rows=session.execute(query)
#    print(rows)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song," - ",row.length)

Faithless Music Matters (Mark Knight Dub)  -  495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query 2
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [16]:
# Create Table 
query="create table artist_byuser "
query= query + " (  userid int, sessionId int, artist text ,song text, User text , "
        #firstname text, lastname text, 
query=query + " primary key ( userid, sessionid))"
print(query)
try:
    session.execute(query)
except Exception as e:
    print(e)


create table artist_byuser  (  userid int, sessionId int, artist text ,song text, User text ,  primary key ( userid, sessionid))
Table 'udacity.artist_byuser' already exists


In [17]:
# Insert the data 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: #artist text,song text, firstname text, lastname text, userid text, sessionId int, itemInSession int,  "
        user = line[1] + " "+ line[4]
        #print((line[0],line[9],user,int(line[10]),int(line[8])))
        query = "insert into artist_byuser ( artist, song, user, userid, sessionId)"
        query = query + "values (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0],line[9],user,int(line[10]),int(line[8])))
        

In [18]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#query="select artist, song, firstname, lastname from artist_byuser where userid = 10 and  sessionid = 182"

query="select artist, song, user from artist_byuser where userid = 10 and  sessionid = 182"

In [19]:

try:
    rows=session.execute(query)
#    print(rows)
    for row in rows:
        print (row.artist, row.song," - ",row.user)
except Exception as e:
    print(e)


Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  -  Sylvie Cruz


In [20]:
# Run the Query

## Query 3    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



In [21]:
data.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88


In [22]:
# Create Table 
query = "create table user_history "
query = query + "( song text, firstName text, lastName text,   "
query = query + " primary key (song, firstName, lastName ))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [23]:
# Insert the data 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: #artist text,song text, firstname text, lastname text, userid text, sessionId int, itemInSession int,  "
        user = line[1] + " "+ line[4]
        #print((line[0],line[9],user,int(line[10]),int(line[8])))
        query = "insert into user_history ( firstName, lastName, song )"
        query = query + "values (%s,%s,%s)"
        session.execute(query, (line[1],line[4],line[9]))
        

In [24]:
query = "select firstName,lastName,song from user_history where song='All Hands Against His Own' "
#query = "select * from user_history where song='All Hands Against His Own' "
try:
    rows=session.execute(query)
#    print(rows)
    for row in rows:
#        print(row)
        print (row.firstname, row.lastname," - ",row.song)
except Exception as e:
    print(e)


Jacqueline Lynch  -  All Hands Against His Own
Sara Johnson  -  All Hands Against His Own
Tegan Levine  -  All Hands Against His Own


### Drop the tables before closing out the sessions

In [25]:
query="drop table artist_session "
try:
    session.execute(query)
except Exception as e:
    print(e)


In [26]:
query="drop table artist_session "
try:
    session.execute(query)
except Exception as e:
    print(e)


Error from server: code=2200 [Invalid query] message="unconfigured table artist_session"


In [27]:
query="drop table user_history "
try:
    session.execute(query)
except Exception as e:
    print(e)



### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()